# Commerce Challenge 1 - Baseline Submission

This notebook provides a simple baseline for **Commerce Challenge 1: Weekly Store-SKU Demand Forecasting**.

**Goal**: Predict `units_sold_next_week` for each SKU-week combination
**Metric**: Root Mean Squared Error (RMSE) - Lower is better

## Instructions:
1. **Replace API credentials** in the first cell with your team's API key and name
2. **Run all cells** to generate and submit baseline predictions
3. **Check the output** for your submission score

This baseline uses only tabular sales data with a simple Random Forest regressor.

In [1]:
from agentds import BenchmarkClient

In [2]:
# 1. Initialize Client and Load Data

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
# from agentds import BenchmarkClient

# # 🔑 REPLACE WITH YOUR CREDENTIALS
# client = BenchmarkClient(
#     api_key="adsb_AJp9fgtjuTRZ4Tk0mNnYDWxG_1759039055",        # Get from your team dashboard
#     team_name="clai"     # Your exact team name
# )


In [3]:
# 🔑 REPLACE WITH YOUR CREDENTIALS
client = BenchmarkClient(
    api_key="adsb_AJp9fgtjuTRZ4Tk0mNnYDWxG_1759039055",        # Get from your team dashboard
    team_name="clai"     # Your exact team name
)

In [4]:
# Cell A — Load CSVs (and optional small preview)

# File names assume you're in the same folder as the CSVs;
# adjust paths if needed.
train_sales   = pd.read_csv("./agent_ds_commerce/sales_history_train.csv")
test_sales    = pd.read_csv("./agent_ds_commerce/sales_history_test.csv")
events_df     = pd.read_csv("./agent_ds_commerce/store_events.csv")
products_df   = pd.read_csv("./agent_ds_commerce/products.csv")

# Ensure core integer types for keys/flags --> convert to int
for df in (train_sales, test_sales):
    for c in ["sku_id", "week", "promo_flag"]:
        if c in df.columns:
            df[c] = df[c].astype(int)

# (Optional) sanity peek to understand your earlier question about price/promos
# sku1_preview = train_sales.query("sku_id == 1").sort_values("week").head(40)
# sku1_preview
len(train_sales), len(test_sales), train_sales.head(2), events_df.head(2), products_df.head(2)


(66000,
 12000,
    sku_id  week  units_sold  price  promo_flag
 0       1     1          51   0.72           0
 1       1     2          41   0.70           0,
    week                                  event_description
 0     1  No large gatherings or city events near the st...
 1     2  Regular trading week with no special events sc...,
    sku_id   category subtype  base_price
 0       1  Beverages   water        0.69
 1       2  Beverages   water        0.69)

In [ ]:
# Cell B — TRAIN features (leak-safe) + label (t+1) + local NLP event-strength
import numpy as np
from typing import Tuple, Dict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Ridge

# ---------------------------
# 1) Week cyclic features
# ---------------------------
def add_week_cycles(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df["week_sin"] = np.sin(2*np.pi*df["week"]/52.0)
    df["week_cos"] = np.cos(2*np.pi*df["week"]/52.0)
    return df

train = add_week_cycles(train_sales)

# ---------------------------
# 2) Products join (safe)
# ---------------------------
prod = products_df.copy()

# Make sure columns exist with correct types
if "category" in prod.columns:
    prod["category"] = prod["category"].astype(str).fillna("unknown")
else:
    prod["category"] = "unknown"

if "brand" in prod.columns:
    prod["brand"] = prod["brand"].astype(str).fillna("unknown")
else:
    prod["brand"] = "unknown"

if "base_price" not in prod.columns:
    prod["base_price"] = np.nan

train = train.merge(prod[["sku_id","category","brand","base_price"]], on="sku_id", how="left")

# ---------------------------
# 3) Signed discount in [-1, +1]
# ---------------------------
def discount_pct_signed(base_price, price) -> float:
    """
    Signed: + means markdown vs base; - means markup vs base.
    Clips to [-1, +1]; returns 0.0 when base_price invalid.
    """
    if base_price is None or np.isnan(base_price) or base_price <= 0 or price is None or np.isnan(price):
        return 0.0
    val = (base_price - price) / base_price
    return float(np.clip(val, -1.0, 1.0))

train["discount_pct"] = [discount_pct_signed(bp, p) for bp, p in zip(train["base_price"], train["price"])]

# ---------------------------
# 4) Global weekly total (train)
# ---------------------------
weekly_total = (
    train.groupby("week", as_index=False)["units_sold"]
    .sum().rename(columns={"units_sold":"total_units_week"})
)
weekly_total["total_units_week_lag1"] = weekly_total["total_units_week"].shift(1)

# ---------------------------
# 5) Per-SKU time-series features (past-only)
# ---------------------------
def build_ts_feats_train(g: pd.DataFrame) -> pd.DataFrame:
    g = g.sort_values("week").copy()  # g is per SKU group
    # short lags for strong short-term memory
    for L in [1,2,3]:
        g[f"lag{L}_units"] = g["units_sold"].shift(L)
    # short rolling mean for stabilization
    g["roll3_mean_units"] = g["units_sold"].shift(1).rolling(3, min_periods=1).mean()
    # price/promo memory
    g["lag1_price"] = g["price"].shift(1)
    g["price_change_pct"] = (g["price"] - g["lag1_price"]) / g["lag1_price"]
    g["lag1_promo"] = g["promo_flag"].shift(1)
    return g

train_feat = train.groupby("sku_id", group_keys=False).apply(build_ts_feats_train)
train_feat = train_feat.merge(weekly_total[["week","total_units_week_lag1"]], on="week", how="left")

# ---------------------------
# 6) Local NLP model: learn event strength (continuous in [-1, +1] + bucket)
#     - train on week-t text to predict week-(t+1) total uplift vs rolling baseline
# ---------------------------
events = events_df.copy()
events["event_description"] = events["event_description"].fillna("").astype(str)

def fit_event_strength_model(
    train_df: pd.DataFrame,
    events_df: pd.DataFrame,
    roll_window: int = 3,
    alpha: float = 2.0,
    low_q: float = 0.33,
    high_q: float = 0.67,
) -> Tuple[pd.DataFrame, Dict[str,float], TfidfVectorizer, Ridge]:
    # weekly totals in train
    wk = train_df.groupby("week", as_index=False)["units_sold"].sum().rename(columns={"units_sold":"weekly_total"})
    wk = wk.sort_values("week").reset_index(drop=True)
    # label = next week's total
    wk["weekly_total_next"] = wk["weekly_total"].shift(-1)
    # rolling baseline for the LABEL (past-only relative to that label)
    baseline = wk["weekly_total_next"].shift(1).rolling(roll_window, min_periods=1).mean()
    wk["uplift"] = (wk["weekly_total_next"] - baseline) / baseline
    wk["uplift"] = wk["uplift"].replace([np.inf, -np.inf], np.nan).fillna(0.0)
    wk["uplift"] = wk["uplift"].clip(-1.0, 1.0)

    # merge week-t text
    data = wk.merge(events_df[["week","event_description"]], on="week", how="left")
    mask = data["uplift"].notna()

    # TF-IDF + Ridge
    tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=2, max_features=5000)
    X = tfidf.fit_transform(data.loc[mask, "event_description"])
    y = data.loc[mask, "uplift"].astype(float)
    ridge = Ridge(alpha=alpha, random_state=42)
    ridge.fit(X, y)

    # predict for ALL weeks in events_df (train + test weeks)
    X_all = tfidf.transform(data["event_description"])
    data["event_strength_pred"] = np.clip(ridge.predict(X_all), -1.0, 1.0)

    # thresholds based on TRAIN predictions only
    train_preds = data.loc[mask, "event_strength_pred"]
    lo_thr = float(train_preds.quantile(low_q))
    hi_thr = float(train_preds.quantile(high_q))

    def bucketize(v):
        if v <= lo_thr:  return "weak"
        if v >= hi_thr:  return "strong"
        return "mild"

    data["event_strength_bucket"] = data["event_strength_pred"].apply(bucketize)

    out = data[["week","event_strength_pred","event_strength_bucket"]].copy()
    return out, {"lo_thr": lo_thr, "hi_thr": hi_thr}, tfidf, ridge

event_strength_all, evt_thr, event_tfidf, event_ridge = fit_event_strength_model(train, events)

# merge event strength into TRAIN features
train_feat = train_feat.merge(event_strength_all, on="week", how="left")

# ---------------------------
# 7) Make label = units at (t+1) for same SKU
# ---------------------------
y_next = train[["sku_id","week","units_sold"]].copy()
y_next["week"] = y_next["week"] - 1
y_next = y_next.rename(columns={"units_sold":"units_sold_next_week"})
train_rows = train_feat.merge(y_next, on=["sku_id","week"], how="inner")

# ---------------------------
# 8) Build TRAIN matrices (encode category/brand top-K)
# ---------------------------
def topk_one_hot(train_series: pd.Series, test_series: pd.Series, k=30, prefix="x"):
    top = train_series.value_counts().index[:k]
    train_c = pd.Categorical(train_series, categories=top)
    test_c  = pd.Categorical(test_series,  categories=top)
    tr = pd.get_dummies(train_c, prefix=prefix)
    te = pd.get_dummies(test_c,  prefix=prefix)
    te = te.reindex(columns=tr.columns, fill_value=0)
    return tr, te  # we'll build TEST later, but need the aligned columns

# Numeric columns (include event strength)
num_cols = [
    "price","promo_flag","discount_pct",
    "week_sin","week_cos",
    "lag1_units","lag2_units","lag3_units",
    "roll3_mean_units",
    "lag1_price","price_change_pct","lag1_promo",
    "total_units_week_lag1",
    "event_strength_pred",   # continuous learned signal
]

# Bucket as one-hots (optional but useful)
evt_tr = pd.get_dummies(train_rows["event_strength_bucket"].fillna("mild"), prefix="evt")

# Product one-hots (derive TRAIN-side encoders here; use aligned cols for TEST later)
cat_tr, _ = topk_one_hot(train_rows["category"], train_rows["category"], k=30, prefix="category")
br_tr,  _ = topk_one_hot(train_rows["brand"],    train_rows["brand"],    k=30, prefix="brand")

X_train_num = train_rows[num_cols].astype(float).fillna(0)
X_train = pd.concat([X_train_num.reset_index(drop=True),
                     evt_tr.reset_index(drop=True),
                     cat_tr.reset_index(drop=True),
                     br_tr.reset_index(drop=True)], axis=1)

y_train = train_rows["units_sold_next_week"].astype(float)

X_train.shape, y_train.shape


C:\Users\shend\AppData\Local\Temp\ipykernel_34252\4147603284.py:79: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  train_feat = train.groupby("sku_id", group_keys=False).apply(build_ts_feats_train)


((64500, 28), (64500,))

In [6]:
# Cell C — TEST features (uses fitted TF-IDF/Ridge + thresholds from Cell B)

# Reuse: products_df, events_df, weekly_total (from train), event_tfidf, event_ridge, evt_thr

# 1) Prep TEST base table
test = add_week_cycles(test_sales.copy())

# Safe product join
test = test.merge(prod[["sku_id","category","brand","base_price"]], on="sku_id", how="left")
test["discount_pct"] = [discount_pct_signed(bp, p) for bp, p in zip(test["base_price"], test["price"])]

# 2) Recompute per-SKU lags on (train ∪ test with units_sold=NaN)
comb = train[["sku_id","week","units_sold","price","promo_flag","week_sin","week_cos",
              "category","brand","base_price","discount_pct"]].copy()
stub = test[["sku_id","week","price","promo_flag","week_sin","week_cos",
             "category","brand","base_price","discount_pct"]].copy()
stub["units_sold"] = np.nan
comb_all = pd.concat([comb, stub], ignore_index=True)

def build_ts_feats_comb(g: pd.DataFrame) -> pd.DataFrame:
    g = g.sort_values("week").copy()
    for L in [1,2,3]:
        g[f"lag{L}_units"] = g["units_sold"].shift(L)
    g["roll3_mean_units"] = g["units_sold"].shift(1).rolling(3, min_periods=1).mean()
    g["lag1_price"] = g["price"].shift(1)
    g["price_change_pct"] = (g["price"] - g["lag1_price"]) / g["lag1_price"]
    g["lag1_promo"] = g["promo_flag"].shift(1)
    return g

comb_feat = comb_all.groupby("sku_id", group_keys=False).apply(build_ts_feats_comb)

# Add global lag and EVENT STRENGTH
comb_feat = comb_feat.merge(weekly_total[["week","total_units_week_lag1"]], on="week", how="left")

# Predict event strength for all weeks using fitted TF-IDF/Ridge
events_all = events_df.copy()
events_all["event_description"] = events_all["event_description"].fillna("").astype(str)

X_evt = event_tfidf.transform(events_all["event_description"])
evt_pred = np.clip(event_ridge.predict(X_evt), -1.0, 1.0)

lo_thr, hi_thr = evt_thr["lo_thr"], evt_thr["hi_thr"]
def bucketize(v):
    if v <= lo_thr:  return "weak"
    if v >= hi_thr:  return "strong"
    return "mild"

events_all = events_all[["week"]].copy().assign(
    event_strength_pred = evt_pred,
    event_strength_bucket = [bucketize(v) for v in evt_pred]
)

comb_feat = comb_feat.merge(events_all, on="week", how="left")

# Slice TEST rows
test_rows = comb_feat[comb_feat["units_sold"].isna()].copy()
test_rows = test_rows.merge(test[["sku_id","week"]], on=["sku_id","week"], how="inner")

# 3) Build TEST matrices aligned with TRAIN columns
num_cols = [
    "price","promo_flag","discount_pct",
    "week_sin","week_cos",
    "lag1_units","lag2_units","lag3_units",
    "roll3_mean_units",
    "lag1_price","price_change_pct","lag1_promo",
    "total_units_week_lag1",
    "event_strength_pred",
]

X_test_num = test_rows[num_cols].astype(float).fillna(0)

# Recreate one-hots with TRAIN's categories (align columns)
evt_te = pd.get_dummies(test_rows["event_strength_bucket"].fillna("mild"), prefix="evt")
evt_te = evt_te.reindex(columns=[c for c in X_train.columns if c.startswith("evt_")], fill_value=0)

# For category/brand, align to TRAIN's one-hot columns
cat_cols = [c for c in X_train.columns if c.startswith("category_")]
br_cols  = [c for c in X_train.columns if c.startswith("brand_")]

def align_one_hot(series: pd.Series, ref_cols, prefix: str):
    # build one-hot on test then reindex to TRAIN columns
    oh = pd.get_dummies(series.astype(pd.CategoricalDtype(
        categories=[c.replace(prefix + "_","") for c in ref_cols]
    )), prefix=prefix)
    return oh.reindex(columns=ref_cols, fill_value=0)

cat_te = align_one_hot(test_rows["category"], cat_cols, prefix="category")
br_te  = align_one_hot(test_rows["brand"],    br_cols,  prefix="brand")

# Final TEST matrix
X_test = pd.concat([X_test_num.reset_index(drop=True),
                    evt_te.reset_index(drop=True),
                    cat_te.reset_index(drop=True),
                    br_te.reset_index(drop=True)], axis=1)

# Ensure the column order matches TRAIN exactly
missing_in_test = [c for c in X_train.columns if c not in X_test.columns]
for c in missing_in_test: X_test[c] = 0
missing_in_train = [c for c in X_test.columns if c not in X_train.columns]
for c in missing_in_train: X_train[c] = 0
X_test = X_test[X_train.columns]

X_train.shape, X_test.shape


C:\Users\shend\AppData\Local\Temp\ipykernel_34252\2367427856.py:30: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  comb_feat = comb_all.groupby("sku_id", group_keys=False).apply(build_ts_feats_comb)


((64500, 28), (12000, 28))

In [1]:
# Cell D — Train baseline model and create submission

from sklearn.ensemble import HistGradientBoostingRegressor
import numpy as np

model = HistGradientBoostingRegressor(
    learning_rate=0.1,
    max_leaf_nodes=63,
    min_samples_leaf=30,
    max_iter=400,
    early_stopping=False,
    random_state=42
)

model.fit(X_train, y_train)

pred = np.clip(model.predict(X_test), 0, None)

submission = test_rows[["sku_id","week"]].copy()
submission["units_sold_next_week"] = pred
submission.to_csv("submission_baseline.csv", index=False)

submission.head(), ("Saved to submission_baseline.csv", submission.shape)


NameError: name 'X_train' is not defined

In [ ]:
# # 2. Tabular-Only Baseline Model and Predictions

# # Select numeric features for baseline
# numeric_features = ['sku_id', 'week', 'price', 'promo_flag']
# print(f"📊 Using features: {numeric_features}")

# # Prepare training data
# X_train = train_sales[numeric_features]
# y_train = train_sales['units_sold']  # Target variable

# # Prepare test data
# X_test = test_sales[numeric_features]

# # Train simple Random Forest regressor baseline
# print("🤖 Training Random Forest regressor...")
# model = RandomForestRegressor(n_estimators=100, random_state=42)
# model.fit(X_train, y_train)

# # Make predictions
# predictions = model.predict(X_test)

# # Create submission file
# submission_df = pd.DataFrame({
#     'sku_id': test_sales['sku_id'],
#     'week': test_sales['week'],
#     'units_sold_next_week': predictions
# })

# # Save predictions
# submission_df.to_csv("commerce_challenge1_predictions.csv", index=False)
# print(f"✅ Predictions saved: {submission_df.shape[0]} predictions")
# print(f"   Preview: {submission_df.head(3)}")
# print(f"   Prediction range: {predictions.min():.2f} to {predictions.max():.2f}")

📊 Using features: ['sku_id', 'week', 'price', 'promo_flag']
🤖 Training Random Forest regressor...
✅ Predictions saved: 12000 predictions
   Preview:    sku_id  week  units_sold_next_week
0       1    45                 73.02
1       1    46                 72.21
2       1    47                 66.83
   Prediction range: 4.38 to 113.81


In [ ]:
# # 3. Submit Predictions

# # Submit predictions to the competition
# print("🚀 Submitting predictions...")

# try:
#     result = client.submit_prediction("Commerce", 1, "commerce_challenge1_predictions.csv")

#     if result['success']:
#         print("✅ Submission successful!")
#         print(f"   📊 Score: {result['score']:.4f}")
#         print(f"   📏 Metric: {result['metric_name']}")
#         print(f"   ✔️  Validation: {'Passed' if result['validation_passed'] else 'Failed'}")
#     else:
#         print("❌ Submission failed!")
#         print(f"   Error details: {result.get('details', {}).get('validation_errors', 'Unknown error')}")

# except Exception as e:
#     print(f"💥 Submission error: {e}")
#     print("🔧 Check your API key and team name are correct!")

# print("\n🎯 Next steps:")
# print("   1. Try incorporating relevant information outside this table!")
# print("   2. Move on to Commerce Challenge 2!")